<a href="https://colab.research.google.com/github/ThanhHung2112/LMS/blob/main/Ensemble_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers
!pip install xgboost

In [ ]:
!pip install -q textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00


In [ ]:
import random
import torch
import torch.optim as optim
import torch.nn as nn
# from transformers import BertModel, BertTokenizer
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare Data


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/CEFR/cefr_combine.csv")
df

,cefr,text
0,A1,"Hi, My name's Leon. I don't speak english, but..."
1,A1,Hi.... From: To: Date: Hi! How are you? My nam...
2,A1,\n\t Dear Anna: I'm Xudong. I'm 33 years old....
3,A1,"\n\t Hello! Nice to meet you. I am fine, than..."
4,A1,\n\t Hi teacher! How are you? My name's Marco...
...,...,...
407551,C2,Light propagating in the vicinity of astrophys...
407552,C2,Future of dentistry has become one of the most...
407553,C2,ï»¿The forests â€“ and suburbs â€“ of Europe a...
407554,C2,Hedge funds are turning bullish on oil once ag...


In [ ]:
df[[
    "smog_index",
    "automated_readability_index",
    "dale_chall_readability_score",
    "difficult_words",
    "linsear_write_formula",
    "gunning_fog",
    "szigriszt_pazos",
    "gutierrez_polini",
    "crawford" ,
    "osman",
    "avg_words"
    ]] = None

In [ ]:
import textstat
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

def avg_words_per_sentence(text):
    sentences = sent_tokenize(text)
    word_count = sum(len(word_tokenize(sentence)) for sentence in sentences)
    if len(sentences) == 0:
        return 0
    return word_count / len(sentences)

df["smog_index"] = df["text"].apply(textstat.smog_index)
df["automated_readability_index"] = df["text"].apply(textstat.automated_readability_index)
df["dale_chall_readability_score"] = df["text"].apply(textstat.dale_chall_readability_score)
df["difficult_words"] = df["text"].apply(textstat.difficult_words)
df["linsear_write_formula"] = df["text"].apply(textstat.linsear_write_formula)
df["gunning_fog"] = df["text"].apply(textstat.gunning_fog)
df["szigriszt_pazos"] = df["text"].apply(textstat.szigriszt_pazos)
df["gutierrez_polini"] = df["text"].apply(textstat.gutierrez_polini)
df["crawford"] = df["text"].apply(textstat.crawford)
df["osman"] = df["text"].apply(textstat.osman)
df["avg_words"] = df["text"].apply(avg_words_per_sentence)
df_original = df

In [ ]:
quantitive = 100
long_text_rows = df_original[df_original["text"].str.split().str.len() > quantitive]

In [ ]:
long_text_rows

,cefr,text,smog_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,szigriszt_pazos,gutierrez_polini,crawford,osman,agv_words,avg_words
54966,A1,"\n\t From : May To: Maria Data : August 28, 2...",7.0,3.9,5.84,5,4.000000,3.62,115.86,52.18,0.5,82.98,None,8.285714
55305,A1,\n\t My name is Erica . I am thirty-two year...,6.9,3.3,5.92,5,5.000000,4.83,117.06,53.42,0.8,87.00,None,12.111111
57892,A1,\n\t Hi ! My name is Jiang Shiying . I'm thir...,5.6,2.5,5.69,3,4.200000,3.08,119.22,54.38,0.2,91.39,None,7.857143
61080,A1,\n\t Hi teacher ! My name's Areej . I'm twent...,6.8,1.6,6.90,6,3.583333,3.38,116.52,55.09,0.3,92.82,None,7.642857
96791,A1,\n\t Hi my name's Marcelo and my favorite day...,5.8,1.3,6.49,4,4.777778,4.64,123.39,58.23,0.5,101.87,None,13.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407551,C2,Light propagating in the vicinity of astrophys...,14.6,14.8,9.46,81,13.000000,13.94,80.69,38.48,3.8,45.37,None,27.533333
407552,C2,Future of dentistry has become one of the most...,15.6,17.5,10.12,151,14.200000,14.79,72.18,31.37,4.5,26.23,None,22.344828
407553,C2,ï»¿The forests â€“ and suburbs â€“ of Europe a...,13.5,17.7,9.78,155,20.000000,13.64,83.12,34.85,3.5,32.93,None,32.840000
407554,C2,Hedge funds are turning bullish on oil once ag...,12.2,12.1,10.16,142,12.400000,10.35,93.62,40.57,2.9,50.85,None,27.964286


# XGBoost


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

df = long_text_rows
df = df.drop(["text", "agv_words"], axis=1)
df = df.sample(frac=1, random_state=200)  # Xáo trộn dữ liệu

# Chuyển đổi các giá trị trong cột "label" thành các giá trị số
level_to_index = {"A1": 0, "A2": 1, "B1": 2, "B2": 3, "C1": 4, "C2": 5}
df["cefr"] = df["cefr"].map(level_to_index)
df

,cefr,smog_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,szigriszt_pazos,gutierrez_polini,crawford,osman,avg_words
405793,4,11.2,11.6,8.60,32,6.000000,8.08,93.68,38.10,2.9,45.21,21.000000
384856,3,12.0,9.7,7.82,23,12.400000,11.87,97.61,46.61,2.6,70.87,18.666667
401598,4,10.3,7.0,8.24,29,8.833333,9.27,105.39,48.08,2.0,73.24,15.230769
389431,3,11.2,9.2,9.34,33,10.666667,10.25,93.08,44.59,3.0,63.64,18.444444
351055,2,6.7,3.8,8.55,10,5.000000,5.20,121.45,53.09,0.5,88.11,11.727273
...,...,...,...,...,...,...,...,...,...,...,...,...
396710,3,14.0,15.5,8.78,26,19.666667,15.32,86.26,40.59,3.0,53.33,32.800000
353915,2,12.0,9.0,7.45,14,12.400000,11.25,98.43,47.51,2.5,73.27,22.800000
249556,1,6.4,5.6,7.27,5,7.666667,7.19,114.71,53.30,1.3,89.66,20.833333
361234,2,9.3,6.0,7.11,17,5.100000,6.99,110.58,47.55,1.4,74.12,12.636364


In [ ]:
X = df.drop(["cefr"], axis=1)
y = df["cefr"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

xgb_model = XGBClassifier(objective="multi:softmax", num_class=6)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6702386751095957


In [ ]:
import joblib

model_filename = "xgboost100.pkl"
joblib.dump(xgb_model, model_filename)


['xgboost100.pkl']

In [ ]:
import textstat
import pandas as pd
from xgboost import XGBClassifier

import nltk
nltk.download('punkt')

import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize

def avg_words_per_sentence(text):
    sentences = sent_tokenize(text)
    word_count = sum(len(word_tokenize(sentence)) for sentence in sentences)
    if len(sentences) == 0:
        return 0
    return word_count / len(sentences)

# Hàm để dự đoán trình độ tiếng Anh dựa trên văn bản đầu vào
def predict_english_level(text, xgb_model):

    smog_index = textstat.smog_index(text)
    automated_readability_index = textstat.automated_readability_index(text)
    dale_chall_readability_score = textstat.dale_chall_readability_score(text)
    difficult_words = textstat.difficult_words(text)
    linsear_write_formula = textstat.linsear_write_formula(text)
    gunning_fog = textstat.gunning_fog(text)
    szigriszt_pazos = textstat.szigriszt_pazos(text)
    gutierrez_polini = textstat.gutierrez_polini(text)
    crawford = textstat.crawford(text)
    osman = textstat.osman(text)
    avg = avg_words_per_sentence(text)

    data = pd.DataFrame({
        "smog_index": [smog_index],
        "automated_readability_index": [automated_readability_index],
        "dale_chall_readability_score": [dale_chall_readability_score],
        "difficult_words": [difficult_words],
        "linsear_write_formula": [linsear_write_formula],
        "gunning_fog": [gunning_fog],
        "szigriszt_pazos": [szigriszt_pazos],
        "gutierrez_polini": [gutierrez_polini],
        "crawford": [crawford],
        "osman": [osman],
        "avg_words": [avg]
    })

    predicted_level = xgb_model.predict(data)
    predicted_probabilities = xgb_model.predict_proba(data)
    # Chuyển kết quả từ số sang trình độ tiếng Anh tương ứng
    level_to_index = {0: "A1", 1: "A2", 2: "B1", 3: "B2", 4: "C1", 5: "C2"}
    predicted_level_text = level_to_index[predicted_level[0]]

    return predicted_level_text, predicted_probabilities


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def predict_average_english_level_for_paragraphs(text):

    paragraphs = text.split("\n")

    level_values = []
    for paragraph in paragraphs:
      if len(paragraph) > 2:
        # print(paragraph)
        predicted_level,prob = predict_english_level(paragraph, xgb_model)
        print(predicted_level)
        if predicted_level is not None:
            level_values.append(predicted_level)


    if not level_values:
        return None

    level_numbers = {'A1': 1,'A2': 2,'B1': 3,'B2': 4,'C1': 5,'C2': 6}
    numeric_levels = [level_numbers[level] for level in level_values]

    # Tính trung bình cộng các giá trị dự đoán
    average_level = sum(numeric_levels) / len(numeric_levels) + 0.5
    # Chuyển giá trị trung bình thành English level cuối cùng
    final_level = [key for key, value in level_numbers.items() if value == round(average_level)]

    return final_level[0],average_level if final_level else None


In [ ]:
text = """
The line graph compares the percentage of Britain households reagularly using their own private transport over a period of 36 years.

Overall, the population of people having one car was the highest and stayed almost unchanged, while the number of household with three or more cars was the least. It is also clear that the figures for no car and two cars dramatically changed.

In 1971, almost half of the population of people in the UK had no car and the households having one car only were the second most populated group. Those with three and more cars had by far the least proportion of population with roughly 2 per cent of them and only 8% of households used two cars.

Over the following 36 years, The percentage of families with one car almost leveled off in about 45% and remained the most popular group, while the less number pf people had no car until in 2007 there were only 25% of them. household started havin two cars with a dramatic rise of around 20%, although three cars and more category had an unconsiderable increase stayed under 8%.
"""

predicted_level,prob = predict_english_level(text, xgb_model)
print("Predicted English Level:", predicted_level)
final_predicted_level,avg = predict_average_english_level_for_paragraphs(text)
print(f"Final Predicted English Level: {final_predicted_level,avg}")


Predicted English Level: B2
B1
A2
B1
B1
Final Predicted English Level: ('B1', 3.25)


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = xgb_model.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(confusion)

accuracy_per_class = {}
for i in range(confusion.shape[0]):
    accuracy_per_class[i] = confusion[i, i] / confusion[i, :].sum()
class_names = ['A1','A2','B1','B2','C1','C2']
for class_label, accuracy in accuracy_per_class.items():
    print(f"Accuracy for class {class_names[class_label]}: {accuracy:.2f}")

Confusion Matrix:
[[  30   12    8    2    0    0]
 [   6  165  162   25    5    0]
 [   1   40 1101  324   42    0]
 [   0   11  291 1184  136    1]
 [   0    0   46  231  255    4]
 [   0    0    0    0    7   17]]
Accuracy for class A1: 0.58
Accuracy for class A2: 0.45
Accuracy for class B1: 0.73
Accuracy for class B2: 0.73
Accuracy for class C1: 0.48
Accuracy for class C2: 0.71


In [ ]:
classification_report_str = classification_report(y_test, y_pred, target_names=class_names)
print("Classification Report:")
print(classification_report_str)

Classification Report:
              precision    recall  f1-score   support

          A1       0.81      0.58      0.67        52
          A2       0.72      0.45      0.56       363
          B1       0.68      0.73      0.71      1508
          B2       0.67      0.73      0.70      1623
          C1       0.57      0.48      0.52       536
          C2       0.77      0.71      0.74        24

    accuracy                           0.67      4106
   macro avg       0.71      0.61      0.65      4106
weighted avg       0.67      0.67      0.67      4106



In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})

unique_labels = df["Actual"].unique()

confusion_matrix_table = pd.crosstab(df["Actual"], df["Predicted"], rownames=["Actual"], colnames=["Predicted"], margins=True)

print(confusion_matrix_table)


Predicted   0    1     2     3    4   5   All
Actual                                       
0          30   12     8     2    0   0    52
1           6  165   162    25    5   0   363
2           1   40  1101   324   42   0  1508
3           0   11   291  1184  136   1  1623
4           0    0    46   231  255   4   536
5           0    0     0     0    7  17    24
All        37  228  1608  1766  445  22  4106


# Neural

In [ ]:
data = []
for index, row in df.iterrows():
    example = {"text": row["text"], "label": row["cefr"]}
    data.append(example)

random.shuffle(data)

In [ ]:
len(data)

1494

In [ ]:
from sklearn.model_selection import train_test_split

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra (90% train, 10% test)
train_data, test_data = train_test_split(data, test_size=0.05, random_state=42)

# Tạo ánh xạ từ trình độ tiếng Anh thành số
level_to_index = {"A1": 0, "A2": 1, "B1": 2, "B2": 3, "C1": 4, "C2": 5}

In [ ]:

# Tạo một mô hình đoán English level
class EnglishLevelPredictor(nn.Module):
    def __init__(self, num_classes):
        super(EnglishLevelPredictor, self).__init__()
        # Load pre-trained BERT model and tokenizer
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # Fully connected layers
        self.fc1 = nn.Linear(768, 256)  # BERT output size is 768
        self.fc2 = nn.Linear(256, num_classes)

        # Activation functions
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_text):
        # Tokenize and encode the input text using BERT
        inputs = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
        outputs = self.bert(**inputs)

        # Get the pooled output (CLS token) from BERT
        pooled_output = outputs['pooler_output']

        # Pass through fully connected layers with ReLU activation
        x = self.fc1(pooled_output)
        x = self.relu(x)
        x = self.fc2(x)

        return x

# Số lớp đầu ra (A1, A2, B1, B2, C1, C2)
num_classes = 6

# Khởi tạo mô hình BERT
model = EnglishLevelPredictor(num_classes)

# Tạo hàm loss (cross-entropy)
criterion = nn.CrossEntropyLoss()

# Tạo optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Huấn luyện mô hình BERT
num_epochs = 5

for epoch in range(num_epochs):
    for example in train_data:
        text = example["text"]
        level = example["label"]

        target = level_to_index.get(level, -1)  # Sử dụng dictionary để ánh xạ

        if target == -1:
            continue

        optimizer.zero_grad()

        output = model(text)

        loss = criterion(output, torch.tensor([target]))

        loss.backward()
        optimizer.step()

In [ ]:
model.eval()  # eval mode
correct = total = 0

with torch.no_grad():
    for example in test_data:
        text = example["text"]
        level = example["label"]
        target = level_to_index.get(level, -1)

        if target == -1:
            continue

        output = model(text)
        _, predicted = torch.max(output, 1)
        total += 1
        correct += (predicted == target).sum().item()

accuracy = (correct / total) * 100
print(f"Validation Accuracy after {num_epochs} epochs of training: {accuracy:.2f}%")

In [ ]:
test_data

In [ ]:
logits_train = []
logits_test = []

with torch.no_grad():
    for example in train_data:
        text = example["text"]
        output = model(text)
        logits_train.append(output.numpy())  # Chuyển đổi thành mảng Numpy

    for example in test_data:
        text = example["text"]
        output = model(text)
        logits_test.append(output.numpy())  # Chuyển đổi thành mảng Numpy

In [ ]:
# Thêm lớp bị thiếu (lớp 4) vào tập dữ liệu
y_train = np.array([level_to_index.get(example["label"], -1) for example in train_data])
y_test = np.array([level_to_index.get(example["label"], -1) for example in test_data])

In [ ]:
print(len(logits_train), len(y_train), ":", len(logits_test), len(y_test))

In [ ]:
y_train

# CEFR predict


In [ ]:
!pip install -q pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 44.8 MB/s eta 0:00:00


In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 8.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("sebastiaan/test-cefr")

In [ ]:
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
validation_df = pd.DataFrame(dataset['validation'])

valid_labels = ["a1", "a2", "b1", "b2", "c1", "c2"]

train_df = train_df[train_df['label'].isin(valid_labels)]
test_df = test_df[test_df['label'].isin(valid_labels)]
validation_df = validation_df[validation_df['label'].isin(valid_labels)]

In [ ]:
print(f"Số mẫu trong tập train: {len(train_df)}")
print(f"Số mẫu trong tập test: {len(test_df)}")
print(f"Số mẫu trong tập validation: {len(validation_df)}")

Số mẫu trong tập train: 120547
Số mẫu trong tập test: 14607
Số mẫu trong tập validation: 15106


In [ ]:
train_df = train_df.rename(columns={'prompt': 'text', 'label': 'label'})
test_df = test_df.rename(columns={'prompt': 'text', 'label': 'label'})
validation_df = validation_df.rename(columns={'prompt': 'text', 'label': 'label'})

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.6 MB/s eta 0:00:00


In [ ]:
!pip install -q accelerate -U
!pip install -q transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    RobertaForSequenceClassification,
    AdamW,
    get_cosine_schedule_with_warmup
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

MODEL_CHECKPOINT = "roberta-base"
SAVE_DIR = "/content/runs-cefr"
SEQ_LEN = 512


class CEFRDataset(Dataset):
    def __init__(self, texts, labels):
        encoder = LabelEncoder()
        self.texts = texts.tolist()
        self.labels = encoder.fit_transform(labels)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoded_text = tokenizer(
            text,
            padding="max_length",
            max_length=SEQ_LEN,
            truncation=True,
            return_tensors="pt"
        )
        encoded_text["input_ids"] = encoded_text["input_ids"].squeeze()
        encoded_text["attention_mask"] = encoded_text["attention_mask"].squeeze()
        label = torch.tensor(label)

        return {
            "input_ids": encoded_text["input_ids"],
            "attention_mask": encoded_text["attention_mask"],
            "labels": label
        }

    def get_labels(self):
        return self.labels


def get_dataset(data):
    # data = pd.read_csv(csv_path)
    return CEFRDataset(data["text"], data["label"])


def train(train_set, valid_set, epochs=10, warmup_size=0.1, lr=1e-3, batch_size=16):
    model = get_model(MODEL_CHECKPOINT)
    optim = AdamW(model.parameters(), lr=lr)
    scheduler = get_scheduler(optim, warmup_size, round(len(train_set) / batch_size * epochs))
    training_args = get_training_args(epochs, batch_size)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=valid_set,
        optimizers=[optim, scheduler],
        compute_metrics=compute_accuracy
    )
    trainer.train()
    trainer.save_model()
    return trainer


def get_model(pretrained_checkpoint):
    model = RobertaForSequenceClassification.from_pretrained(
        pretrained_checkpoint,
        num_labels=6
    )
    return model.to(device)


def get_scheduler(optimizer, warmup_size, total_steps):
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=round(total_steps*warmup_size),
        num_training_steps=total_steps
    )
    return scheduler


def get_training_args(epochs, batch_size):
    return TrainingArguments(
        output_dir=SAVE_DIR,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        logging_steps=50,
        fp16=True,
        evaluation_strategy="epoch",
        eval_accumulation_steps=1
    )


def compute_accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}


Using device: cuda


In [ ]:
model = get_model('roberta-base')

(…)co/roberta-base/resolve/main/config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
train_set = get_dataset(train_df[:5000])
valid_set = get_dataset(validation_df[:300])

In [ ]:
len(train_set)

5000

In [ ]:
trainer = train(train_set, valid_set, epochs=10, warmup_size=0.2, lr=1e-5, batch_size=8)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.583200,1.566587,0.290000
2,1.325900,1.365824,0.426667
3,1.080100,1.049033,0.606667
4,0.798700,1.186505,0.560000
5,0.634800,1.102578,0.576667
6,0.460400,1.153907,0.650000
7,0.308200,1.297470,0.633333
8,0.160800,1.428175,0.646667
9,0.141300,1.483150,0.646667
10,0.158000,1.514982,0.650000


In [ ]:
test_set = get_dataset(test_df)
predictions = trainer.predict(test_set)
predictions[2]

{'test_loss': 1.8430473804473877,
 'test_accuracy': 0.5984117204080236,
 'test_runtime': 174.3717,
 'test_samples_per_second': 83.769,
 'test_steps_per_second': 10.472}

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")


In [ ]:

# Create a custom classifier model using PyTorch Lightning
class CEFRClassifier(pl.LightningModule):
    def __init__(self, num_classes):
        super(CEFRClassifier, self).__init__()
        self.bert = bert_model
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 256, dtype=torch.float32)
        self.fc2 = nn.Linear(768, 256, dtype=torch.float32)
        self.fc3 = nn.Linear(16, 256, dtype=torch.float32)  # Update this based on the number of other features
        self.fc4 = nn.Linear(256, num_classes, dtype=torch.float32)  # Updated to match the number of classes
        self.softmax = nn.Softmax(dim=1) # Modify the output size to match the number of classes

    def forward(self, input_ids, attention_mask, pos_input_ids, pos_attention_mask, other_features):
        bert_output = self.bert(input_ids, attention_mask)
        bert_hidden_state = bert_output.last_hidden_state
        pooled_output = bert_output.pooler_output

        pos_tags_embedding = self.bert(pos_input_ids, pos_attention_mask).last_hidden_state[:, 0, :]

        x1 = self.relu(self.fc1(bert_hidden_state[:, 0, :]))
        x2 = self.relu(self.fc2(pooled_output))
        x3 = self.relu(self.fc3(other_features))

        combined_features = torch.cat((x1, x2, x3), dim=1)
        logits = self.fc4(combined_features)
        logits = self.softmax(logits)

        return logits

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, pos_input_ids, pos_attention_mask, other_features, labels = batch
        logits = self(input_ids, attention_mask, pos_input_ids, pos_attention_mask, other_features)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, labels)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


In [ ]:
# data
df = pd.read_csv("/content/cefr.csv")

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

encoded_train_texts = tokenizer(
    list(train_df['text']),
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='pt'
)
encoded_test_texts = tokenizer(
    list(test_df['text']),
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='pt'
)

encoded_train_pos_tags = tokenizer(list(train_df['pos_tags']), padding=True, truncation=True, return_tensors='pt')
encoded_test_pos_tags = tokenizer(list(test_df['pos_tags']), padding=True, truncation=True, return_tensors='pt')
labels_encoder = LabelEncoder()
train_labels = torch.tensor(labels_encoder.fit_transform(train_df['label']))
test_labels = torch.tensor(labels_encoder.transform(test_df['label']))
other_features_train = train_df[['flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index', 'coleman_liau_index', 'automated_readability_index', 'dale_chall_readability_score', 'difficult_words', 'linsear_write_formula', 'gunning_fog', 'text_standard', 'fernandez_huerta', 'szigriszt_pazos', 'gutierrez_polini', 'crawford', 'gulpease_index', 'osman']].to_numpy().astype('float32')
other_features_test = test_df[['flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index', 'coleman_liau_index', 'automated_readability_index', 'dale_chall_readability_score', 'difficult_words', 'linsear_write_formula', 'gunning_fog', 'text_standard', 'fernandez_huerta', 'szigriszt_pazos', 'gutierrez_polini', 'crawford', 'gulpease_index', 'osman']].to_numpy().astype('float32')


In [ ]:
# Create dataloaders
train_data = torch.utils.data.TensorDataset(
    encoded_train_texts.input_ids,
    encoded_train_texts.attention_mask,
    encoded_train_pos_tags.input_ids,
    encoded_train_pos_tags.attention_mask,
    torch.tensor(other_features_train),
    train_labels
)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)

test_data = torch.utils.data.TensorDataset(
    encoded_test_texts.input_ids,
    encoded_test_texts.attention_mask,
    encoded_test_pos_tags.input_ids,
    encoded_test_pos_tags.attention_mask,
    torch.tensor(other_features_test),
    test_labels
)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=8)

# Instantiate the model
model = CEFRClassifier(num_classes=6)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, precision=32)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:

import os

# Define the max_split_size_mb as an environment variable
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "128"  # Adjust the value as needed

# Rest of your code


In [ ]:
import gc
del trainer
gc.collect()

5224

In [ ]:
# Training loop using PyTorch Lightning Trainer
trainer.fit(model, train_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type      | Params
--------------------------------------
0 | bert    | BertModel | 109 M 
1 | relu    | ReLU      | 0     
2 | fc1     | Linear    | 196 K 
3 | fc2     | Linear    | 196 K 
4 | fc3     | Linear    | 4.4 K 
5 | fc4     | Linear    | 1.5 K 
6 | softmax | Softmax   | 0     
--------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
439.527   Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
# Evaluation
model.eval()
test_results = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, pos_input_ids, pos_attention_mask, other_features, labels = batch
        logits = model(input_ids, attention_mask, pos_input_ids, pos_attention_mask, other_features)
        predicted_labels = torch.argmax(logits, dim=1)
        test_results.append(predicted_labels)

test_results = torch.cat(test_results, dim=0)
accuracy = accuracy_score(test_labels, test_results)
print(f'Accuracy: {accuracy}')